In [1]:
import pandas as pd
from pymongo import MongoClient
import numpy as np

In [2]:
def get_mongo_database(db_name, host='localhost',
                       port=27017, username=None, password=None):
    """ Get named database from MongoDB with/out authentication """
    # make Mongo connection with/out authentication
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s'%(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)

    return conn[db_name]

def mongo_coll_to_dicts(dbname='test', collname='test',
                        query={}, del_id=True, **kw): 

    db = get_mongo_database(dbname, **kw)
    res = list(db[collname].find(query))

    if del_id:
        for r in res:
            r.pop('_id')

    return res

def mongo_to_dataframe(db_name, collection, query={},
                       host='localhost', port=27017,
                       username=None, password=None, no_id=True):
    """ create a dataframe from mongodb collection """
    db = get_mongo_database(db_name, host, port, username, password)
    cursor = db[collection].find(query)
    df =  pd.DataFrame(list(cursor))

    if no_id: 
        del df['_id']

    return df

def dataframe_to_mongo(df, db_name, collection,
                       host='localhost', port=27017,
                       username=None, password=None):
    """ save a dataframe to mongodb collection """
    db = get_mongo_database(db_name, host, port, username, password)
    records = df.to_dict('records')
    db[collection].insert_many(records)

In [3]:
DB_NOBEL_PRIZE='nobel_prize'
COLL_winners='winners'
COLL_winners_clean='winners_clean'

In [4]:
df=mongo_to_dataframe(DB_NOBEL_PRIZE,COLL_winners)

In [5]:
#1 : fixing mixed datatypes
#Replace empty strings with NaN where it makes sense to count them as missing data
df.replace('', np.nan, inplace=True)
df.name=df.name.str.replace('\*','')
df.name=df.name.str.strip()

#2 : specific cleaning
#remove duplicates identified as winners with born_in dates
df=df[df.born_in.isnull()]
del df['born_in']

#identify institututions as winners with no genre nor date of birth
df.loc[(df.gender.isnull())&(df.date_of_birth.isnull()),'receiver_nature']='institution'

#cleaning due to scrapping/storing failure
df=df[~((df.gender.isnull())&(df['receiver_nature']!='institution'))]
df.drop_duplicates(inplace=True)

#delete duplicates identified as winners with same name and year of attribution
df=df[~(df.duplicated('name')&df.duplicated('year'))]

In [6]:
#missing categories
df.loc[df.name=="Alexis Carrel",'category']=u'Physiology or Medicine'
df.loc[df.category.isnull(),'category']=u'Physiology or Medicine'
df.loc[df.name=="Amartya Sen",'category']=u'Economics'
#missing place_of_birth
df.loc[df.name=="Val Fitch","place_of_birth"]="Merriman"
df.loc[df.name=="John Robert Vane","place_of_birth"]="Tardebigge"
df.loc[df.name=="Carlos Filipe Ximenes Belo","place_of_birth"]="Baucau"
df.loc[df.name=="James A. Mirrlees","place_of_birth"]="Minnigaff"
#missing place of death
df.loc[df.name=="Charles J. Pedersen","place_of_death"]="Salem"
df.loc[df.name=="Archer John Porter Martin","place_of_death"]="Llangaren"

In [7]:
#isolate df containing dates
with_death_date=df[df.date_of_death.notnull()]
#obtain a filter identifying errors (=NaT)
bad_dates=pd.isnull(pd.to_datetime(with_death_date.date_of_death,errors='coerce'))
with_bad_date=with_death_date[bad_dates]

#no errors both for birth and death so let's convert to datetime
df.date_of_death=pd.to_datetime(df.date_of_death)
df.date_of_birth=pd.to_datetime(df.date_of_birth)

#additional cleaning based on award age analysis
df.loc[df.name==u'Ragnar Granit','year']=1967
df.loc[df.name==u'Artturi Ilmari Virtanen','year']=1945
df.loc[df.name==u'Frans Eemil Sillanpää','year']=1939

#add an award_age column using pd.DatetimeIndex
df['award_age']=df.year -  pd.DatetimeIndex(df.date_of_birth).year

#replace NaT by 0 to export via pymongo
df.loc[df.date_of_birth.isnull(),'date_of_birth']=0
df.loc[df.date_of_death.isnull(),'date_of_death']=0

#export to mongo
dataframe_to_mongo(df,DB_NOBEL_PRIZE,COLL_winners_clean)

In [8]:
df.head()

,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year,receiver_nature,award_age
1,Peace,Belgium,1970-01-01,1970-01-01,NaN,https://en.wikipedia.org/wiki/Institut_de_Droi...,Institut de Droit International,NaN,NaN,"Institut de Droit International , Peace, 1904",1904,institution,NaN
17,Peace,Belgium,1829-07-26 00:00:00,1912-10-06,male,https://en.wikipedia.org/wiki/Auguste_Marie_Fr...,Auguste Beernaert,Ostend,Lucerne,"Auguste Beernaert , Peace, 1909",1909,NaN,80.0
20,Peace,Argentina,1931-11-26,1970-01-01,male,https://en.wikipedia.org/wiki/Adolfo_P%C3%A9re...,Adolfo Pérez Esquivel,Buenos Aires,NaN,"Adolfo Pérez Esquivel , Peace, 1980",1980,NaN,49.0
22,Chemistry,Argentina,1906-09-06,1987-12-02,male,https://en.wikipedia.org/wiki/Luis_Federico_Le...,Luis Federico Leloir,Paris,Catamarca Province,"Luis Federico Leloir , Chemistry, 1970",1970,NaN,64.0
24,Physiology or Medicine,Argentina,1887-04-10 00:00:00,1971-09-21,male,https://en.wikipedia.org/wiki/Bernardo_Houssay,Bernardo Houssay,Buenos Aires,Buenos Aires,"Bernardo Houssay , Physiology or Medicine, 1947",1947,NaN,60.0


In [88]:
mini_bio=pd.read_json("../scrapping/nobel/nobel_winners/winners_mini_bio.json")
db['mini_bio'].insert_many(mini_bio.to_dict(orient='records'))

In [89]:
#convert mongo source to df
mini_bio=mongo_to_dataframe(DB_NOBEL_PRIZE,'mini_bio')
#only consider links which appear in the main dataset
mini_bio_clean=mini_bio[mini_bio.link.isin(df.link.unique())]
#drop duplicates
mini_bio_clean=mini_bio_clean[~mini_bio_clean.duplicated('link')]
#merge with main
full_data=pd.merge(df, mini_bio_clean, how='outer', on='link')
#replace NaN that trigger errors in json
df.award_age[df.award_age.isnull()]=0
df.gender[df.gender.isnull()]='institution'
df.place_of_birth[df.place_of_birth.isnull()]='institution'
df.place_of_death[df.place_of_death.isnull()]='not applicable'
df.receiver_nature[df.receiver_nature.isnull()]='people'
#insert full in mongo
db['full_data'].insert_many(full_data.to_dict(orient='records'))